In [3]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import pymongo
from Config import api_key

ModuleNotFoundError: No module named 'Config'

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# Define database and collection
db = client.wwf_species
collection = db.species

In [ ]:
#Creating a list of endangered species by scraping 2 pages of https://www.worldwildlife.org/species/directory?page=1

endangered_animals = []
for i in range(1,3):

   url = "https://www.worldwildlife.org/species/directory?page=" + str(i)
   response = requests.get(url)
   time.sleep(1)

# Scrape page into Soup

   soup = bs(response.text, 'html')

   results = soup.find_all('td', class_ ='keep')

   for result in results:
      animal = result.text
      endangered_animals.append(animal)
print(endangered_animals)

In [ ]:
# Going to each species web page and scraping for image, and other information pertaining to status, population,
# scientific name, description and image
base_url = "https://www.worldwildlife.org/species"
for animal in endangered_animals:
    # Error handling
    try:
# Getting to each species URL    
        animal_url = animal.replace(" ","-") 
        url = base_url +"/"+ animal_url
        response = requests.get(url)
# Scraping for information for each species        
        soup = bs(response.text, 'html')
        overview_species = soup.find('div', id='overview')
        all_overview = overview_species.find_all('div', class_='container')
        species_status = all_overview[0].text
        species_population = all_overview[1].text
        species_scientific_name = all_overview[2].text
        species_description = soup.find('div', class_='wysiwyg lead').text
        image_path = soup.find('img',class_='centered')
        # Run only if title, price, and link are available
        if (all_overview and species_description and image_path):
            print(image_path)
            species_image = image_path['src']
        
# Creating a dictionary and putting it all the scraped information together        
            species_post = {
                'Species Name' : animal,
                'Status' : species_status,
                'Population' : species_population,
                'Scientific Name' : species_scientific_name,
                'Species Description' : species_description,
                'Species Image URL' : species_image
            }
# Inserting the record in the database collection    
            collection.insert_one(species_post)
    except Exception as e:
        print(e)

In [ ]:
# Display items in MongoDB collection
specie_listings = db.species.find()

for specie in specie_listings:
    print(specie)


In [ ]:

# Search for articles that mention animals

Urls = []

 

for animals in endangered_animals:

    query =  animals

 

    # Build query URL

 

    query_url = nytimesurl + 'q='+query+'&api-key='+api_key+'&sort=newest'

#   query_url

# Request articles

    articles = requests.get(query_url).json()

 

# The "response" property in articles contains the actual articles

# list comprehension.

    articles_list = [article for article in articles["response"]["docs"]]

    for article in articles_list:     

        Urls.append= article["web_url"]

 

# Dictionary to be inserted as a MongoDB document

articles_dict = {

    'endagered_animal': animals,

    'url': [Urls]

        }

collection.insert_one(articles_dict)

 